In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.utils import to_categorical

In [2]:
#Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# ResNet50: This is a pre-trained convolutional neural network (CNN) architecture included in the keras.applications module.

In [6]:
# reshape the data
X_train = x_train.reshape((x_train.shape[0], 28, 28))
X_test = x_test.reshape((x_test.shape[0], 28, 28))
# change the type to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# convert data to 3 channels since pretrained model ResNet50 expected image data with 3 channels and
X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)


X_train = tf.image.resize(X_train, (32, 32)).numpy()
X_test = tf.image.resize(X_test, (32, 32)).numpy()


Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3) (with "channels_last" data format) or (3, 224, 224) (with "channels_first" data format). It should have exactly 3 inputs channels, and width and height should be no smaller than 32. E.g. (200, 200, 3) would be one valid value.

In [7]:
#Load ResNet50 WITHOUT top layer (feature extractor)
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
resnet.trainable = False  # freeze ResNet

In [9]:
# Extract features
features_train = resnet.predict(X_train, batch_size=64, verbose=1)
features_test = resnet.predict(X_test, batch_size=64, verbose=1)

print("Feature shape:", features_train.shape)

# Flatten features for feed forward network (FFN)
features_train = features_train.reshape((features_train.shape[0], -1))
features_test = features_test.reshape((features_test.shape[0], -1))

print("Flattened feature shape:", features_train.shape)

938/938 ━━━━━━━━━━━━━━━━━━━━ 318s 336ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 49s 311ms/step
Feature shape: (60000, 1, 1, 2048)
Flattened feature shape: (60000, 2048)


In [10]:
# Define a simple FNN classifier
inputs = Input(shape=(features_train.shape[1],))
x = Dense(32, activation='relu')(inputs)
x = Dense(16, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Another way to define a model with pretrained model
#### Model uses ResNet50 as a feature extractor (pretrained on ImageNet).Feed those extracted features into a basic Feedforward Neural Network (FNN) for classifying MNIST digits.

model = Sequential()\
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))\
model.add(Dense(16, activation='relu'))\
model.add(Dense(10, activation='softmax'))\
model.layers[0].trainable=False\
model.summary()\

In [11]:
# Train the classifier
model.fit(features_train, y_train, epochs=10, batch_size=64, validation_data=(features_test, y_test))

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.7648 - loss: 0.7649 - val_accuracy: 0.9222 - val_loss: 0.2614
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9377 - loss: 0.1969 - val_accuracy: 0.9473 - val_loss: 0.1724
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9483 - loss: 0.1626 - val_accuracy: 0.9438 - val_loss: 0.1845
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9533 - loss: 0.1405 - val_accuracy: 0.9519 - val_loss: 0.1529
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9589 - loss: 0.1275 - val_accuracy: 0.9575 - val_loss: 0.1408
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9614 - loss: 0.1193 - val_accuracy: 0.9489 - val_loss: 0.1697
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9632 - loss: 0.1118 - val_accuracy: 0.9573 - val_loss: 0.1369
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9652 - loss: 0.1048 - val_accuracy

In [12]:
# Evaluate
test_loss, test_acc = model.evaluate(features_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9536 - loss: 0.1572
Test Accuracy: 0.9594
